In [1]:
import sys

In [2]:
from get_page import *
from get_urls import *

# Sample of a single page

In [3]:
URL = "https://medlineplus.gov/ency/article/001316.htm"

In [4]:
finished_page = get_page_entry(URL)

In [5]:
finished_page

['medlineplus.gov/ency/article/001316.htm',
 'Malignant teratoma of the mediastinum',
 'A teratoma is a type of cancer that contains one or more of the three layers of cells found in a developing baby (embryo). These cells are called germ cells. A teratoma is one type of germ cell tumor.',
 [['Causes',
   [('p',
     'Malignant mediastinal teratoma occurs most often in young men in their 20s or 30s. Most malignant teratomas can spread throughout the body, and have spread by the time of diagnosis.'),
    ('p', 'Blood cancers are often associated with this tumor, including:'),
    ('ul',
     ['Acute myelogenous leukemia (AML)',
      'Myelodysplastic syndromes (group of bone marrow disorders)'])]],
  ['Symptoms',
   [('p', 'Symptoms may include:'),
    ('ul',
     ['Chest pain or pressure',
      'Cough',
      'Fatigue',
      'Limited ability to tolerate exercise',
      'Shortness of breath'])]],
  ['Exams and Tests',
   [('p',
     'The health care provider will perform a physical e

# Find and save a copy of each articles URL

In [6]:
filename = "data/all_urls"

In [7]:
#create_urls_list(filename)

### Progress bar

In [8]:
def progress_bar(total, current):
    
    message = "{:>4}/{}".format(current, total)
    
    bar_length = 50
    progress   = int((current / total) * bar_length)
    remaining  = bar_length - progress

    bar = "[" + ("#"*progress) + ("."*remaining) + "] - " + message

    print(bar, end="\r")

# Using all retrieved URLs

In [9]:
url_prefix = "https://medlineplus.gov/ency/"

In [10]:
# open file
file  = open("{}.txt".format(filename), 'r')
lines = file.readlines()
file.close()
print("{} urls retrieved".format(len(lines)))

3560 urls retrieved


In [11]:
num_to_read = len(lines)
all_pages = []
for i in range(len(lines)):
    line = lines[i]
    
    whole_url = (url_prefix + line).strip("\n")
    page = get_page_entry(whole_url)
    all_pages.append(page)
    
    progress_bar(num_to_read, i+1)

print("\n{} pages read".format(len(all_pages)))

[##################################################] - 3560/3560
3560 pages read


In [12]:
sys.getsizeof(all_pages)/1000 # bytes / 1000 = kb

30.112

<hr>

# Generate sql entries

In [14]:
# finished_page = [page_URL, page_title, page_summary, finished_sections, symptoms_section]
prepared_pages = []
symptoms_join = {}
page_sections = []
section_id = 0
for i in range(1, len(all_pages)+1):
    page = all_pages[i-1]
    
    # sql data
    id_ = i
    ency_url     = page[0]
    page_title   = page[1].replace("'", "''").replace("\n", '').strip(' ')
    page_summary = str(page[2])
    if type(page_summary) == type(None):
        page_summary = ''
    page_summary = page_summary.replace("'", "''")
        
    other_names  = ""
    
    
    for section in page[3]:
        if section[0] == "Alternative Names":
            other_names = section[1][0][1]
        
        elif section[0] == "Symptoms":
            for item in section[1]:
                if item[0] == 'ul':
                    for j in range(len(item[1])):
                        symptom = item[1][j].replace("\n", '').replace(".", "")
                        symptom = symptom.lower()
                        
                        #if symptom in symptoms_join: 
                        if symptom in symptoms_join:
                            if id_ not in symptoms_join[symptom]:
                                symptoms_join[symptom].append(id_)
                        else:
                            symptoms_join[symptom] = [id_]
                            
#                else:
#                    symptom = item[1].strip("\n").strip('.')
#                    symptom = symptom.lower()
#
#                    if symptom in symptoms_join and id_ not in symptoms_join[symptom]: 
#                        symptoms_join[symptom].append(id_)
#                    else:
#                        symptoms_join[symptom] = [id_]
                    
                        
        elif section[0] == "Causes":
            page_sections.append([section_id, id_, section[0], section[1]])
            section_id += 1
                        
    prepared_pages.append([id_, ency_url, page_title, page_summary, other_names])

## symptoms matching

In [15]:
def merge_symptoms(gen_name, name_, rarity=None, qualifier_type=None):

    all_symptoms[gen_name]['other_names'   ].append(name_)
    
    if qualifier_type != None:
        all_symptoms[gen_name][qualifier_type].append(rarity)
    
    
    for key in all_symptoms[name_]['pages'].keys():
        # add just this qualifier (rarity)
        # ---------------------------------
        if rarity != None:
            if key == 'none': qualifier = rarity
            else:             qualifier = "{} {}".format(rarity, key)
        else:
            qualifier = key
        
        # add together
        # -------------
        if qualifier in all_symptoms[gen_name]['pages']:
            all_symptoms[gen_name]['pages'][qualifier] += all_symptoms[name_]['pages'][key]
        else:
            all_symptoms[gen_name]['pages'][qualifier] = all_symptoms[name_]['pages'][key]

        all_symptoms[gen_name]['count'] += len(all_symptoms[name_]['pages'][key])
        
        
    # check for duplicate connections
    # --------------------------------
    all_page_connections = []
    for key in all_symptoms[gen_name]['pages'].keys():
        page_connections = all_symptoms[gen_name]['pages'][key]
        to_remove = []
        for i in range(len(page_connections)):
            if page_connections[i] in all_page_connections: to_remove.append(page_connections[i])
            else:                                           all_page_connections.append(page_connections[i])
            
        for id_ in to_remove:
            all_symptoms[gen_name]['pages'][key].remove(id_)
            all_symptoms[gen_name]['count'] -= 1

In [16]:
# symptoms matching
'''
symptoms_join

pages = {'mild': [34], 'common': [450, 567]}
'''
join_keys = list(symptoms_join.keys())

all_symptoms = {}

for i in range(1, len(join_keys)+1):
    key = join_keys[i-1]
    
    primary_name = key
    primary_name = primary_name#.replace("'", "''") 
    primary_name = primary_name#.replace(',', '\,')
    
    
    # joiner table
    # -------------
    pages_ = {'none':[]}
    count  = 0
    for j in range(len(symptoms_join[key])):
        page_id = symptoms_join[key][j]
        if page_id in pages_['none']: print(page_id, primary_name)
        pages_['none'].append(page_id)
        count += 1
    
    # symptoms table
    # ---------------
    symptom_ = {
        "id"               : i,
        "primary_name"     : primary_name,
        "count"            : count,
        "other_names"      : [],
        "known_rarities"   : [],
        "known_severities" : [],
        "pages"            : pages_
    }
    
    if primary_name in all_symptoms:
        print(primary_name, i)
        print(all_symptoms[primary_name])
    else:
        all_symptoms[primary_name] = symptom_

In [17]:
severities = ['mild', 'severe', 'in severe cases', 'long-term', 'chronic', 'throbbing']
rarities   = ['sometimes', 'common', 'uncommon', 'rare', 'in women', 'in children']

In [18]:
'''
another thing i could do would be check the other_names for everything for matches

check for lists of things

'''

symptom_keys = list(all_symptoms.keys())

rare_count = 0
seve_count = 0
tech_count = 0
non_parenth_count = 0
for i in range(len(symptom_keys)):
    name_ = symptom_keys[i]
    
    # rarity
    # -------
    rarity_changed = False
    for rarity in rarities:
        if rarity in name_:
            # replace the rarity, get rid of extra spaces, get rid of leftover parenthesis
            gen_name = name_.replace(rarity, '').replace('()', '').replace('( )', '')
            gen_name = gen_name.strip(' ')
            
            if gen_name in all_symptoms.keys():
                rare_count += 1
                rarity_changed = True
                
                #print(name_, gen_name)
                merge_symptoms(gen_name, name_, rarity, 'known_rarities')
                
    
    # severity
    # ---------
    severity_changed = False
    for severity in severities:
        if severity in name_:
            # replace the rarity, get rid of extra spaces, get rid of leftover parenthesis
            gen_name = name_.replace(severity, '').replace('()', '').replace('( )', '')
            gen_name = gen_name.strip(' ')
            
            if gen_name in all_symptoms.keys():
                seve_count += 1
                severity_changed = True
                
                #print(name_, gen_name)
                merge_symptoms(gen_name, name_, severity, 'known_severities')
                
                        
    # technical name
    # ---------------
    tech_name = False
    if '(' in name_ and ')' in name_ and ',' not in name_:
        start = 0
        end   = 0
        for j in range(len(name_)):
            if name_[j] == '(': start = j+1
            if name_[j] == ')': end   = j
        inner = name_[start:end].lower()
        gen_name = inner.strip(' ')

        if gen_name in all_symptoms.keys():
            tech_count += 1
            tech_name = True
            
            #print(name_, "|", gen_name)
            merge_symptoms(gen_name, name_)
            
    
    # check non-parenthesis text
    # ---------------------------
    # only do this if rarity and severity didn't find anything
    non_parenth = False
    if not rarity_changed and not severity_changed:
        if '(' in name_ and ')' in name_:
            start = 0
            end   = 0
            for j in range(len(name_)):
                if name_[j] == '(': start = j+1
                if name_[j] == ')': end   = j
            inner = name_[:start-1].lower()
            gen_name = inner.strip(' ')    

            if gen_name in all_symptoms.keys():
                non_parenth_count += 1
                non_parenth = True

                #print(name_, "|", gen_name)
                merge_symptoms(gen_name, name_)

                  
    
    if   rarity_changed:   all_symptoms.pop(name_)
    elif severity_changed: all_symptoms.pop(name_)
    elif tech_name:        all_symptoms.pop(name_)
    elif non_parenth:      all_symptoms.pop(name_)        
        
        
print(rare_count)
print(seve_count)
print(tech_count)
print(non_parenth_count)

changes = sum([rare_count, seve_count, tech_count, non_parenth_count])
print(changes)

# original num: 8942

print( "{} + {} = {}".format(len(all_symptoms), changes, (len(all_symptoms)+changes)))

30
54
244
611
939
8085 + 939 = 9024


In [19]:
# unique symptoms 12500
# unique symptoms 12465 - rarities
# unique symptoms 12415 - both
# unique symptoms 12138 - tech names

# 8942
# 8086

print("unique symptoms: {}".format(len(all_symptoms)))
print("----------------------")

#test = dict(reversed(sorted(all_symptoms.items(), key=lambda item: item[1]['count'])))
test = dict(sorted(all_symptoms.items(), key=lambda item: item[1]['count'], reverse=True))

test_keys = list(test.keys())
limit     = 10

for i in range(len(test_keys)):
    if i > limit: 
        print("...")
        break
    key = test_keys[i]
    print("{:>5} - {}".format(test[key]['count'], key))

unique symptoms: 8085
----------------------
  246 - headache
  225 - fever
  203 - nausea and vomiting
  195 - diarrhea
  188 - seizures
  170 - fatigue
  163 - abdominal pain
  153 - vomiting
  150 - shortness of breath
  127 - dizziness
  120 - weakness
...


# write to sql

In [20]:
def sql_array(list_):
    if len(list_) == 0: return "'{}'"

    line = "'{"
    for i in range(len(list_)):
        entry = list_[i]
        
        # cleaning
        # ---------
        entry = entry.strip(' ').strip('\n')
        entry = entry.replace("'", "''")
        entry = entry.replace('"', '') 
        entry = entry.replace(',', '\,')
        
        line += '"{}", '.format(entry)

    return line[:-2] + "}'"

### pages

In [21]:
insert_statement = "INSERT INTO page_entry (id, ency_url, title, summary, other_names)\nVALUES\n"
pages_sql = []


for i in range(len(prepared_pages)):
    page = prepared_pages[i]
    
    other_names = page[4].split(';')
    array_names = sql_array(other_names)
    
    line = "({}, '{}', '{}', '{}', {}),\n".format(page[0], page[1], page[2], page[3], array_names)
    insert_statement += line


insert_statement = insert_statement[:-2] + ";\n\n"

### symptoms and joiner table

In [22]:
#all_symptoms['fatigue']

In [23]:
'''
    symptom_ = {
        "id"               : i,
        "primary_name"     : primary_name,
        "count"            : count,
        "other_names"      : [],
        "known_rarities"   : [],
        "known_severities" : [],
        "pages"            : pages_
    }
'''
join_keys = list(all_symptoms.keys())

symptoms_sql = "INSERT INTO symptoms (id, primary_name, count, other_names, known_rarities, known_severities)\nVALUES\n"
joiner_sql   = "INSERT INTO page_symptoms (page_id, symptom_id, rarity, severity)\nVALUES\n"
for i in range(1, len(join_keys)+1):
    
    key = join_keys[i-1]
    
    symptom_ = all_symptoms[key]
    
    primary_name = key
    primary_name = primary_name.replace("'", "''") 
    #primary_name = primary_name.replace(',', '\,')
    
    # symptoms table
    # ---------------
    id_   = symptom_['id']
    count = symptom_['count']
    other_names      = sql_array(symptom_['other_names'])
    known_rarities   = sql_array(symptom_['known_rarities'])
    known_severities = sql_array(symptom_['known_severities'])
    
    
    line = "({}, '{}', {}, {}, {}, {}),\n".format(id_, primary_name, count, other_names, known_rarities, known_severities)
    symptoms_sql += line
    
    
    # joiner table
    # -------------
    qualifiers = list(symptom_['pages'].keys())
    for j in range(len(qualifiers)):
        symptom_id = id_
        qualifier = qualifiers[j]
        
        if qualifier in known_rarities:
            rarity   = qualifier
            severity = ""
        elif qualifier in known_severities:
            rarity   = ""
            severity = qualifier
        else:
            rarity   = ""
            severity = ""
            
        for k in range(len(symptom_['pages'][qualifier])):
            page_id = symptom_['pages'][qualifier][k]

            line = "({}, {}, '{}', '{}'),\n".format(page_id, symptom_id, rarity, severity)
            joiner_sql += line
            

symptoms_sql = symptoms_sql[:-2] + ";\n\n"
joiner_sql   = joiner_sql[:-2]   + ";\n\n"

### write all to sql file

In [24]:
output_sql = "data/sd_data.sql"

# erase current file
open(output_sql, "w").close()

# write statements
f = open(output_sql, "a")

f.write(insert_statement)
f.write(symptoms_sql)
f.write(joiner_sql)

f.close()

<hr>

# Inspect section title frequencies

In [20]:
# finished_page = [page_URL, page_title, page_summary, finished_sections, symptoms_section]
section_titles_dict = {}
symptoms_dict       = {}
parenthesis_dict    = {}

for page in all_pages:
    for section in page[3]:
        title = section[0]
        if "Review Date" in title: title = "Review Date"
        if title in section_titles_dict: section_titles_dict[title] += 1
        else:                            section_titles_dict[title]  = 1
            
        if title == "Symptoms":
            for item in section[1]:
                if item[0] == 'ul':
                    for j in range(len(item[1])):
                        symptom = item[1][j].replace("\n", '').replace(".", "")
                        symptom = symptom.lower()
                        
                        if symptom in symptoms_dict: symptoms_dict[symptom] += 1
                        else:                        symptoms_dict[symptom]  = 1
        
                        if '(' in symptom:
                            start = 0
                            end   = 0
                            for i in range(len(symptom)):
                                if symptom[i] == '(': start = i+1
                                if symptom[i] == ')': end   = i
                            inner = symptom[start:end].lower()
                            if inner in parenthesis_dict: parenthesis_dict[inner] += 1
                            else:                         parenthesis_dict[inner]  = 1
        

In [21]:
print("unique section titles {}".format(len(section_titles_dict)))
print("-------------------------\n")

for key in list(section_titles_dict.keys()):
    print("{:>5} - {}".format(section_titles_dict[key], key))


unique section titles 51
-------------------------

  596 - How the Test is Performed
  575 - How to Prepare for the Test
  561 - How the Test will Feel
  591 - Why the Test is Performed
  554 - Normal Results
  587 - What Abnormal Results Mean
  757 - Risks
 2944 - Alternative Names
  417 - Patient Instructions
 2471 - Images
 3500 - References
 3560 - Review Date
 3560 - 
 1815 - Causes
 1810 - Symptoms
 1466 - Exams and Tests
 1483 - Treatment
  272 - Support Groups
 1902 - Outlook (Prognosis)
 1109 - Possible Complications
 1692 - When to Contact a Medical Professional
 1040 - Prevention
  457 - Home Care
  255 - What to Expect at Your Office Visit
  234 - Description
  233 - Why the Procedure is Performed
  203 - Before the Procedure
  231 - After the Procedure
  383 - Considerations
   57 - First Aid
  274 - Poisonous Ingredient
  258 - Where Found
  281 - Before Calling Emergency
  281 - Poison Control
  280 - What to Expect at the Emergency Room
  222 - Information
   49 - Func

<hr>

In [1]:
# unique symptoms 8942

#print("unique symptoms {}".format(len(symptoms_dict)))
#print("---------------------\n")

#symptoms_dict = dict(reversed(sorted(symptoms_dict.items(), key=lambda item: item[1])))

#for key in list(symptoms_dict.keys()):
#    print("{:>5} - {}".format(symptoms_dict[key], key))

In [2]:
#print("unique parenthesis {}".format(len(parenthesis_dict)))
#print("-----------------------\n")

#parenthesis_dict = dict(reversed(sorted(parenthesis_dict.items(), key=lambda item: item[1])))

#also_symptoms = {}
#for key in list(parenthesis_dict.keys()):
#    if key.lower() in symptoms_dict:
#        also_symptoms[key.lower()] = parenthesis_dict[key]
#    print("{:>5} - {}".format(parenthesis_dict[key], key))

In [3]:
'''
unique parenthesis 1446
-----------------------

   58 - shock
   48 - jaundice
   45 - decreased level of consciousness and lack of responsiveness
   42 - seizures
   37 - palpitations
   36 - malaise
   33 - cyanosis
   32 - lack of responsiveness
   31 - rare
   28 - severe
   23 - esophagus
   21 - may also cause breathing difficulty
   18 - photophobia
   16 - which may also cause breathing difficulty
   15 - convulsions
   15 - food pipe
   13 - pallor
   12 - edema
   12 - nystagmus
   11 - common
   11 - extremely low blood pressure
   11 - tinnitus
   10 - sometimes
   10 - unresponsiveness
   10 - ph balance
   10 - dilated
    9 - in women
    9 - watery, bloody
    9 - incontinence
    9 - necrosis
    8 - decreased level of consciousness
    8 - agitation and confusion
    8 - petechiae
    7 - holes
    7 - from inhalation
    7 - euphoria
    7 - decreased level of alertness
    7 - stridor
    7 - in children
    7 - ptosis
    7 - ataxia
    7 - icterus
    7 - tachycardia
    7 - occasionally
    7 - hallucinations
    6 - urinary retention
    6 - low libido
    6 - micrognathia
    6 - shaking
    6 - not understandable
    6 - sputum
    6 - ulcers
    6 - lesions
    6 - in severe cases
    6 - seeing or hearing things that aren't there
    6 - develops rapidly
    6 - stupor
    6 - ascites
    6 - may contain blood
    6 - moon face
    5 - uncommon
    5 - gynecomastia
    5 - opisthotonos
    5 - may be bloody
    5 - scoliosis
    5 - amenorrhea
    5 - tachypnea
    5 - unconsciousness
    5 - lethargy
    5 - dysuria
    5 - with mycoplasma pneumonia
    5 - dyspnea
    5 - wide
    5 - vertigo
    5 - regurgitation
    4 - ataxia, seen commonly in children
    4 - may stop completely
    4 - hypothermia
    4 - enlarged
    4 - in some cases
    4 - decreased awareness, sleepiness, confusion
    4 - gerd
    4 - angina
    4 - common, sometimes with blood
    4 - in men
    4 - central obesity
    4 - 1/2 inch or 1 centimeter or more wide
    4 - microcephaly
    4 - yellow skin
''';

In [4]:
#print("also symptoms {}".format(len(also_symptoms)))
#print("---------------------\n")

#for key in list(also_symptoms.keys()):
#    print("{:>5} - {}".format(also_symptoms[key], key))

In [ ]:
'''
also symptoms 117
---------------------

   48 - jaundice
   42 - seizures
    1 - yellow skin and whites of the eyes
   58 - shock
    1 - weakness
    2 - unsteady gait
   36 - malaise
    1 - short stature
    1 - sleeping difficulty
    7 - hallucinations
    2 - impotence
    1 - leaking urine
    1 - watery eyes
    1 - nasal congestion
   37 - palpitations
    6 - stupor
    2 - altered level of consciousness
    1 - too much or too little acid in the blood, which leads to damage in all of the body organs
   15 - convulsions
    1 - clammy skin
   13 - pallor
    1 - paleness
    3 - possible bleeding in the stomach and intestines
    2 - wheezing
    4 - raynaud phenomenon
    1 - hyperactivity
    1 - tissue death
    1 - hard stools
    2 - general ill feeling
    1 - growth failure
    1 - fainting
    9 - incontinence

''';